In [ ]:
"""
dataset = load_dataset("text", data_files="medical_corpus.txt")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


dataset = TextDataset(tokenized_text, tokenizer,block_size=128)
"""


In [ ]:
# loading the model & peft weights from huggingface hub

peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = model.to(device)


In [ ]:
element_counts = Counter(list(ans_dict.values()))

for element, count in element_counts.items():
    print(f"{element}: {count}")

In [ ]:
acc,f1_macro=[],[]
for i in range(5):
    ans_dict=model_pubmed_evaluator(model=model,tokenizer=tokenizer,config=pubmed_config,id_=i)
    with open("/home/medical-llama/pubmed-pred/base.json", "w") as json_file:
        json.dump(ans_dict, json_file)
    accuracy,f1=pub_scores("/home/medical-llama/pubmed-pred/base.json")
    acc.append(accuracy)
    f1_macro.append(f1)

print(acc)
print(f1_macro)

print("Accuracy: ",np.mean(acc))
print("F1 Macro: ",np.mean(f1_macro))